# Capítulo 11: Método Simples de Hückel para moléculas orgânicas conjugadas

Livro: Aprendendo Química com Python, Rodrigo Q. de Albuquerque & Gerd B. da Rocha, 2021, Amazon Book.

E-Mail: learn.chem.python@gmail.com

### Versão do Python usada

In [124]:
import sys
print(sys.version)

3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) 
[GCC 9.3.0]


### Caixa de sessão interativa 40: Montagem da matriz de Hückel e sua diagonalização

In [125]:
import numpy as np
connect = np.array([[0,1],[1,0]])
connect

array([[0, 1],
       [1, 0]])

In [126]:
# Definição do número de átomos da molécula 
natoms = len(connect)   
print(natoms)

2


In [127]:
# Definição dos parâmetros alfa e beta
alpha, beta = -6.553, -2.734  

In [128]:
# Define uma matriz alfa
alpha_mat = alpha*np.identity(len(connect),float) 

In [129]:
# Define uma matriz beta
beta_mat = beta*connect            

In [130]:
alpha_mat

array([[-6.553, -0.   ],
       [-0.   , -6.553]])

In [131]:
beta_mat

array([[-0.   , -2.734],
       [-2.734, -0.   ]])

In [132]:
# Matriz Hamiltoniana de Hückel
H = alpha_mat + beta_mat  
print(H)

[[-6.553 -2.734]
 [-2.734 -6.553]]


In [133]:
# Para importar um módulo específico do SciPy
from scipy import linalg as la 
e_MO, C_MO = la.eig(H)
print("Autovalores = ", e_MO)
print("Autovetores = ", C_MO)

Autovalores =  [-3.819+0.j -9.287+0.j]
Autovetores =  [[ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]


In [134]:
idx = np.argsort(e_MO)
e_MO = np.real(e_MO[idx])
C_MO = np.real(C_MO[:,idx])
print("Autovalores ordenados = ", e_MO) 
print("Autovetores ordenados = ", C_MO)

Autovalores ordenados =  [-9.287 -3.819]
Autovetores ordenados =  [[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]


### Caixa de sessão interativa 41: Cálculo da energia eletrônica da molécula de eteno

In [135]:
nel_pi = 2   # Número de elétrons pi
norbs = 2    # Número de orbitais pi
norb_occ = int(nel_pi / 2)  # Número de orbitais pi ocupados
print(norb_occ)

1


In [136]:
# Número de orbitais pi virtuais
norb_virt = norbs - norb_occ 
print(norb_virt)

1


In [137]:
# Número de ocupação dos orbitais moleculares
size_H = 2
num_occup_MOs = np.zeros(size_H) 
for i in range(norb_occ):
    num_occup_MOs[i] = 2

In [138]:
print("Número de ocupação dos Orbitais Moleculares = ", num_occup_MOs)    

Número de ocupação dos Orbitais Moleculares =  [2. 0.]


In [139]:
ener_pi = 0.0
for j in range(norb_occ):
    ener_pi += num_occup_MOs[j]*e_MO[j]
print("Energia (eV) = ", ener_pi)

Energia (eV) =  -18.573999999999998


### 

In [140]:
# Cálculo das cargas atômicas pi
q_pi = np.zeros(size_H, dtype=float) # Carga atômica pi

for i in range(norbs):
    soma = 0.0
    for j in range(norbs):
        num = C_MO[i][j]
        soma += (num*num)*num_occup_MOs[j]
    
    q_pi[i] = 1 - soma

print("Cargas atômicas líquidas pi = ", q_pi)

Cargas atômicas líquidas pi =  [2.22044605e-16 2.22044605e-16]


### 

In [141]:
# Cálculo da matriz de ordem de ligação Pi
# Matriz ordem de ligação Pi
bond_pi = np.zeros((size_H, size_H), dtype=float) 

for i in range(norbs):    
    for j in range(norbs):
        soma = 0.0
        for k in range(norbs):            
            soma += C_MO[i][k]*C_MO[j][k]*num_occup_MOs[k]
        
        bond_pi[i][j] = soma

print("Matriz ordem de ligação Pi = ", bond_pi)

Matriz ordem de ligação Pi =  [[1. 1.]
 [1. 1.]]


### 

In [142]:
# Cálculos espectroscópicos
e_HOMO = e_MO[norb_occ-1]
e_LUMO = e_MO[norb_occ]
gap_HOMO_LUMO = e_LUMO - e_HOMO
e_ioniz = -e_HOMO
c = 3.0e8  # Velocidade da luz, em m/s
h = 6.62608e-34  # Constante de Planck, em J*s
eV = 1.60218e-19  # Conversão de eV para J
comp_onda = (h*c)/(gap_HOMO_LUMO*eV)
print("gap HOMO-LUMO (eV) = ", gap_HOMO_LUMO)
print("Comprimento de onda da transição HOMO --> LUMO (nm) = ", comp_onda*1.0e9)
print("Energia de Ionização (eV)", e_ioniz)

gap HOMO-LUMO (eV) =  5.467999999999999
Comprimento de onda da transição HOMO --> LUMO (nm) =  226.90189225811878
Energia de Ionização (eV) 9.286999999999999


### 

In [143]:
# Calculando índices de reatividade
# Suscetibilidade eletrofílica
S_eletro = np.zeros(norbs, dtype=float)
# Suscetibilidade nucleofílica
S_nucleo = np.zeros(norbs, dtype=float)
# Suscetibilidade radicalar
S_radic = np.zeros(norbs, dtype=float) 

for i in range(norbs):
    soma = 0.0
    for j in range(norb_occ):
        num = C_MO[i][j]
        soma += num*num*2.0/e_MO[j]
    
    S_eletro[i] = soma

for i in range(norbs):
    soma = 0.0
    for j in range(norb_occ+1):
        num = C_MO[i][j]
        soma += num*num*2.0/e_MO[j]
    
    S_nucleo[i] = soma

S_radic = S_eletro + S_nucleo

print("Sucetibilidades eletrofílicas = ", S_eletro) 
print("Sucetibilidades nucleofílicas = ", S_nucleo) 
print("Sucetibilidades radicalares = ", S_radic) 

Sucetibilidades eletrofílicas =  [-0.1076774 -0.1076774]
Sucetibilidades nucleofílicas =  [-0.36952605 -0.36952605]
Sucetibilidades radicalares =  [-0.47720345 -0.47720345]
